# Center of the small Quantum group

## Introduction

This notebook will show how to compute the center of the small quantum group, as described in the two papers by Annach Lachowska and You Qi:

https://arxiv.org/abs/1604.07380v3

https://arxiv.org/abs/1703.02457v3

Let $G$ be a complex simple Lie group, and let $P$ be a parabolic subgroup. Then we consider the cotangent bundle of the associated partial flag variety:
$$
    \tilde{\mathcal N}_P :=T^*(G/P)
$$
We are then interested in computing
$$HH^s(\tilde{\mathcal N}_P)\cong \bigoplus_{i+j+k=s}H^i(\tilde{\mathcal N}_P,\wedge^jT\tilde{\mathcal N}_P)^k$$
This can be computed by using the BGG resolution. We define the following module:

$$M_j^k = \bigoplus_r \operatorname{Sym}^{j-r+k/2}\mathfrak u_P\otimes \wedge^r\mathfrak g\otimes \wedge^{j-r}\mathfrak n_P$$

Let $\Delta\colon\mathfrak p\to \mathfrak g\oplus \mathfrak u_P\otimes\mathfrak n_P$ be given by the inclusion in the first component and in the second component by the adjoint action (after identifying $\operatorname{End}(\mathfrak n_P)$ with $\mathfrak u_P\otimes \mathfrak n_P$). Then $\Delta$ induces a map $M_{j-1}^k\to M_j^k$. We define the module

$$E_j^k = M_j^k\big/\Delta(M_{j-1}^k)$$

Then the cohomology of the BGG resolution of $E_j^k$ in degree $i$ with respect to a dominant weight $\mu$ computes the multiplicity of $\mu$ of $H^i(\tilde{\mathcal N}_P,\wedge^jT\tilde{\mathcal N}_P)^k$.

## Example

We begin by loading the required packages. Change `sys.path.append('../bggcomplex')` as required. The `module quantum_center_module_v2` has additional dependency `tqdm` (recent versions), you can install it by running the following command in a cell:

    !pip install tqdm -U

In [1]:
import sys
sys.path.append('../bggcomplex')

from fast_module import BGGCohomology
from bggcomplex import BGGComplex
from quantum_center_module_v2 import *

Now let us compute a simple example and set $G=A2$. We will compute $H^0(\tilde{\mathcal N}_0,\wedge^2T\tilde{\mathcal N}_0)^{-2}$. Here the subscript $0$ indicates the principal block. The principal block corresponds to the parameter `subset=[]`. The other parameters are `s,i,j,k`, and they satisfy `i+j+k=s` and `j>=i`.

If we want to change the parabolic subalgebra $P$ then we can change `subset` to e.g. `subset=[1]` for 
$$\mathfrak p = \mathfrak b\oplus\mathbb C\langle e_1\rangle=\mathbb C\langle e_1,f_1,f_2,f_{12} \rangle,\qquad \mathfrak u_P = \mathbb C\langle e_{12},e_2\rangle,\qquad \mathfrak n_P = \mathbb C\langle f_{12},f_2\rangle$$

In [2]:
BGG = BGGComplex('A2')
s = 0
i = 0
j = 2
k = s-i-j
print('i+j=%d, j-i=%d, k=%d\n'%(i+j,j-i,k))

subset = []
cohom = BGGCohomology(BGG, Mijk(BGG,i,j,k,subset=subset), coker=Tijk_basis(BGG, i,j,k,subset=subset))
cohom.cohomology_LaTeX(i=i,complex_string=r'\tilde{\mathcal N}_0,\wedge^2T\tilde{\mathcal N}_0)^{-2}')

i+j=2, j-i=2, k=-2



<IPython.core.display.Math object>

## Bigraded tables
The program can also produce bigraded tables. We decompose impose a bigrading on the total cohomology $HH^s(\tilde{\mathcal N}_P)$ by $a=i+j$ and $b = j-i$. Since the computation can take a while a loading bar is provided. 

Due to the $\mathfrak{sl}_2$ symmetry in the bigraded table, we only actually need to compute half of it. We can choose to either compute the whole table, or only half if it through the `half_only` parameter. It's default value is `False`. 

The bigraded table can be rendered in LaTeX through the `display_ab_dic` function, which takes an additional parameter `extend_half` which computes the second half of the table from the first half through symmetry considerations. The produced LaTeX code can easily be copied.

The parameter `compact` determines how the weight decomposition of the cohomology is displayed. E.g. with `compact=False` it would display $\mathbb C\oplus L(\alpha_1+\alpha_2)^2$ and with `compact=True` it gives $\mathbb C L_{1,1}^2$.

In [3]:
# the parameters we actually want to change
BGG = BGGComplex('B4')
subset=[]
s = 0

# compute only half of the table, extend by symmetry
half_only = True
extend_half = half_only

# Display in full form
compact = False

ab_dic = dict()
with tqdm(all_abijk(BGG,s=s,subset=subset,half_only=half_only)) as inner_pbar:
    with tqdm(leave=None) as outer_pbar:
        for a,b,i,j,k in inner_pbar:
            map_pbar
            inner_pbar.set_description('i+j= %d, j-i = %d'%(a,b))
            coker = Tijk_basis(BGG, i,j,k,subset=subset,pbar=outer_pbar)
            outer_pbar.set_description('Initializing cohomology')
            cohom = BGGCohomology(BGG, Mijk(BGG,i,j,k,subset=subset), 
                                  coker=coker)
            outer_pbar.set_description('Computing cohomology')
            cohom_string = cohom.cohomology_LaTeX(i=i,only_strings=True,compact=compact)
            ab_dic[(a,b)]=cohom_string
display_ab_dic(ab_dic,extend_half = extend_half)

KeyboardInterrupt: 